In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
pd.options.plotting.backend = "plotly"

In [151]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-01-17 18:48:10,084 - Authenticating...
2023-01-17 18:48:10,714 - Successfully authenticated


In [152]:
super_clarion(clarion)

In [153]:
from gioutils.tools_clarion import get_history_plotter_book

In [132]:
data, p = get_history_plotter_book(clarion, bq, stir_as_swap=False)

2023-01-17 18:34:55,929 - Requesting data
2023-01-17 18:35:13,259 - Request complete
2023-01-17 18:35:13,275 - Response Count Summary - DFRAME:338
2023-01-17 18:35:13,946 - Response for '3M23 Comdty' contains securityError {'securityError': {'source': '19435:rsfrdsvc1', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19435]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-17 18:35:13,947 - Response for '3M24 Comdty' contains securityError {'securityError': {'source': '19435:rsfrdsvc1', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19435]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-17 18:35:13,947 - Response for '3M25 Comdty' contains securityError {'securityError': {'source': '19435:rsfrdsvc1', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19435]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-17 18:35:13,948 - Response for '3M26 Comdty' contains securityError {'securityError': {'source': '19435:

In [155]:
from gioutils.ezutils import get_irbt_ts
from gioutils.clarion_ext import Runner

In [156]:
bbg_instruments = [
    'RatesTreasuryFuturesOption',
    'RatesShortFuturesOption',
    'RatesTreasuryFutures',
    'RatesShortFutures'
]

In [157]:
end_date = today()
offset = "20y"
start_date = end_date - parse_offset(offset)

In [171]:
def get_swaps_from_postions(data):
    data = data.query('TradeType == "RatesSwap" or as_swap == True')
    
    swaps = []
    for index, swap in data.iterrows():
        
        period = swap['hpperiod'].lower().split('x')
        
        i = clarion.instruments.RatesSwap(
            currency=swap['LocalCurrency'],
            instrument_name=swap['Underlying'],
            strike='a',
            tenor=period[1] if len(period) > 1 else period[0],
            swap_type=swap['PayReceive'][:3],
            start_date=period[0] if len(period) > 1 else None,
            clearing_house='LCH',
            id=str(swap['id'])
        )
        swaps.append(i)
    return swaps

In [172]:
swaps = get_swaps_from_postions(data)

In [173]:
swaps

[id: 275800481, tradeType: RatesSwap, {'Currency': 'EUR', 'InstrumentName': 'EUR ESTER', 'Strike': 'a', 'Tenor': '2y', 'SwapType': 'Rec', 'StartDate': '5y', 'ClearingHouse': 'LCH'},
 id: 275800531, tradeType: RatesSwap, {'Currency': 'EUR', 'InstrumentName': 'EUR ESTER', 'Strike': 'a', 'Tenor': '2y', 'SwapType': 'Pay', 'StartDate': '3y', 'ClearingHouse': 'LCH'},
 id: 275806141, tradeType: RatesSwap, {'Currency': 'EUR', 'InstrumentName': 'EUR ESTER', 'Strike': 'a', 'Tenor': '1y', 'SwapType': 'Rec', 'StartDate': '6m', 'ClearingHouse': 'LCH'}]

In [174]:
strike = clarion.metric("strike", "Strike", "Any")

In [175]:
runner = Runner(clarion=clarion)

In [176]:
res = runner.run(
    metrics=strike,
    data_list=swaps,
    market_mode="reval",
    start_date=start_date,
    end_date=end_date,
    # system_time=system_time,
    # polling_count=10,
    # polling_interval=1
)

2023-01-17 18:49:35,905 - Requesting data
2023-01-17 18:49:45,087 - Request complete
2023-01-17 18:49:45,189 - Response Count Summary - DFRAME:7698, HAS_ERROR:1080


In [49]:
res.pivot(index='period', columns='id', values='strike')

id          EUR ESTER A 3Yx2Y Pay  EUR ESTER A 5Yx2Y Rec  \
period                                                     
2013-02-27               1.135453               1.950666   
2013-02-28               1.133356               1.945674   
2013-03-01               1.091068               1.927478   
2013-03-04               1.112929               1.925268   
2013-03-05               1.134172               1.957085   
...                           ...                    ...   
2023-01-10               2.437179               2.455584   
2023-01-11               2.308744               2.342718   
2023-01-12               2.266404               2.309417   
2023-01-13               2.173684               2.261685   
2023-01-16               2.159711               2.262847   

id          EUR ESTER A 6Mx1Y Rec  
period                             
2013-02-27               0.149132  
2013-02-28               0.125171  
2013-03-01               0.123685  
2013-03-04               0.126578  
2013-03-05               0.148867  
...                           ...  
2023-01-10               3.293165  
2023-01-11               3.264998  
2023-01-12               3.184071  
2023-01-13               3.177013  
2023-01-16               3.177860  

[2566 rows x 3 columns]

In [177]:
res

id periodType          description     period    strike
0     275800481      daily  EUR EUR ESTER 5y 2y 2013-02-27  1.950666
1     275800481      daily  EUR EUR ESTER 5y 2y 2013-02-28  1.945674
2     275800481      daily  EUR EUR ESTER 5y 2y 2013-03-01  1.927478
3     275800481      daily  EUR EUR ESTER 5y 2y 2013-03-04  1.925268
4     275800481      daily  EUR EUR ESTER 5y 2y 2013-03-05  1.957085
...         ...        ...                  ...        ...       ...
7693  275806141      daily  EUR EUR ESTER 6m 1y 2023-01-10  3.293165
7694  275806141      daily  EUR EUR ESTER 6m 1y 2023-01-11  3.264998
7695  275806141      daily  EUR EUR ESTER 6m 1y 2023-01-12  3.184071
7696  275806141      daily  EUR EUR ESTER 6m 1y 2023-01-13  3.177013
7697  275806141      daily  EUR EUR ESTER 6m 1y 2023-01-16  3.177860

[7698 rows x 5 columns]

In [105]:
hist_curves = res.pivot(index='period', columns='id', values='strike')

In [110]:
hist_curves.dropna()

id          EUR ESTER A 3Yx2Y Pay  EUR ESTER A 5Yx2Y Rec  \
period                                                     
2013-02-27               1.135453               1.950666   
2013-02-28               1.133356               1.945674   
2013-03-01               1.091068               1.927478   
2013-03-04               1.112929               1.925268   
2013-03-05               1.134172               1.957085   
...                           ...                    ...   
2023-01-10               2.437179               2.455584   
2023-01-11               2.308744               2.342718   
2023-01-12               2.266404               2.309417   
2023-01-13               2.173684               2.261685   
2023-01-16               2.159711               2.262847   

id          EUR ESTER A 6Mx1Y Rec  
period                             
2013-02-27               0.149132  
2013-02-28               0.125171  
2013-03-01               0.123685  
2013-03-04               0.126578  
2013-03-05               0.148867  
...                           ...  
2023-01-10               3.293165  
2023-01-11               3.264998  
2023-01-12               3.184071  
2023-01-13               3.177013  
2023-01-16               3.177860  

[2566 rows x 3 columns]

In [82]:
bbg_data = d.query(f'TradeType in {bbg_instruments}')

In [102]:
bbg_history = bq.bdh(
    securities=bbg_data['cticker'].drop_duplicates(),
    fields=['yld_ytm_mid'],
    start_date=start_date,
    end_date=end_date,
)

In [103]:
yld = bbg_history.pivot(index='date', columns='security', values='yld_ytm_mid')

In [107]:
hist_data = pd.concat([yld, hist_curves], axis=1).ffill().dropna()

In [119]:
for index, group in data.query('Category == "rates"').groupby('Strategy', sort=False):
    break

In [ ]:
tickers = i['cticker'] if i['as_swap']

In [129]:
hist_data.loc[:, group['cticker'] , group['Description']]

RX1 Comdty  RX1 Comdty  RX1 Comdty
2018-05-04       0.069       0.069       0.069
2018-05-07       0.052       0.052       0.052
2018-05-08       0.087       0.087       0.087
2018-05-09       0.087       0.087       0.087
2018-05-10       0.070       0.070       0.070
...                ...         ...         ...
2023-01-11       1.844       1.844       1.844
2023-01-12       1.791       1.791       1.791
2023-01-13       1.783       1.783       1.783
2023-01-16       1.792       1.792       1.792
2023-01-17       1.710       1.710       1.710

[1228 rows x 3 columns]

In [127]:
group['cticker']

14    RX1 Comdty
15    RX1 Comdty
16    RX1 Comdty
Name: cticker, dtype: object

In [122]:
hist_data

0E6 Comdty  0Q8 Comdty  3M1 Comdty  ED6 Comdty  ER6 Comdty  \
2018-05-04       2.925       2.735       3.633       2.925      -0.110   
2018-05-07       2.925       2.760       3.634       2.925      -0.115   
2018-05-08       2.940       2.770       3.653       2.940      -0.110   
2018-05-09       2.960       2.795       3.680       2.960      -0.105   
2018-05-10       2.960       2.815       3.666       2.960      -0.105   
...                ...         ...         ...         ...         ...   
2023-01-11       3.660       3.100       4.216       3.660       3.010   
2023-01-12       3.555       3.000       4.125       3.555       2.925   
2023-01-13       3.675       3.110       4.178       3.675       2.905   
2023-01-16       3.675       3.110       4.178       3.675       2.900   
2023-01-17       3.605       3.045       4.178       3.605       2.785   

            FF2 Comdty  RX1 Comdty  SFR12 Comdty  SFR2 Comdty  SFR4 Comdty  \
2018-05-04       1.835       0.069         2.730        2.210        2.465   
2018-05-07       1.835       0.052         2.755        2.160        2.440   
2018-05-08       1.840       0.087         2.765        2.160        2.450   
2018-05-09       1.840       0.087         2.790        2.160        2.465   
2018-05-10       1.840       0.070         2.810        2.175        2.485   
...                ...         ...           ...          ...          ...   
2023-01-11       4.635       1.844         2.850        4.885        4.725   
2023-01-12       4.595       1.791         2.725        4.845        4.685   
2023-01-13       4.595       1.783         2.785        4.855        4.760   
2023-01-16       4.595       1.792         2.785        4.855        4.760   
2023-01-17       4.590       1.710         2.760        4.845        4.715   

            SFR7 Comdty  SFR8 Comdty  EUR ESTER A 3Yx2Y Pay  \
2018-05-04        2.680        2.700               0.674299   
2018-05-07        2.675        2.710               0.667090   
2018-05-08        2.685        2.720               0.694897   
2018-05-09        2.710        2.745               0.703043   
2018-05-10        2.730        2.765               0.686261   
...                 ...          ...                    ...   
2023-01-11        3.400        3.100               2.308744   
2023-01-12        3.295        3.000               2.266404   
2023-01-13        3.415        3.110               2.173684   
2023-01-16        3.415        3.110               2.159711   
2023-01-17        3.340        3.045               2.159711   

            EUR ESTER A 5Yx2Y Rec  EUR ESTER A 6Mx1Y Rec  
2018-05-04               1.129585              -0.306407  
2018-05-07               1.118444              -0.305110  
2018-05-08               1.161288              -0.302205  
2018-05-09               1.168851              -0.298581  
2018-05-10               1.149340              -0.298796  
...                           ...                    ...  
2023-01-11               2.342718               3.264998  
2023-01-12               2.309417               3.184071  
2023-01-13               2.261685               3.177013  
2023-01-16               2.262847               3.177860  
2023-01-17               2.262847               3.177860  

[1228 rows x 15 columns]

In [120]:
group


id isIntraday Category  Strategy               Ticker  \
14  277023182.0       True    rates  MM:BUNDS          RXH3 Comdty   
15         12.0      False    rates  MM:BUNDS  RXG3P 137.00 Comdty   
16          7.0      False    rates  MM:BUNDS          RXH3 Comdty   

      Underlying                   Description LocalCurrency  Notional  \
14  EUX BUND 10Y                BUND EUX MAR23           EUR     -50.0   
15  EUX BUND 10Y  BUND EUX FEB23xMAR23 137 Put           EUR     500.0   
16  EUX BUND 10Y                BUND EUX MAR23           EUR    -100.0   

                     TradeType  ...     cticker rticker   cticker_aged  \
14        RatesTreasuryFutures  ...  RX1 Comdty      rx  GTDEM10Y Govt   
15  RatesTreasuryFuturesOption  ...  RX1 Comdty      rx  GTDEM10Y Govt   
16        RatesTreasuryFutures  ...  RX1 Comdty      rx  GTDEM10Y Govt   

    hpcurve hpperiod hpperiod_aged    delta_rate  \
14      NaN  2mx113m       1mx113m   6400.759417   
15      NaN  2mx113m       1mx113m  19892.498913   
16      NaN  2mx113m       1mx113m  12801.518834   

                          string                      string_aged  as_swap  
14  [BBG RX1 Comdty YLD_YTM_MID]  [BBG GTDEM10Y Govt YLD_YTM_MID]    False  
15  [BBG RX1 Comdty YLD_YTM_MID]  [BBG GTDEM10Y Govt YLD_YTM_MID]    False  
16  [BBG RX1 Comdty YLD_YTM_MID]  [BBG GTDEM10Y Govt YLD_YTM_MID]    False  

[3 rows x 33 columns]

In [ ]:
data = get_clarion_positions()

2023-01-13 00:53:05,974 - Requesting data
2023-01-13 00:53:22,897 - Request complete
2023-01-13 00:53:22,907 - Response Count Summary - DFRAME:32


In [ ]:
delta_tol = 100
delta_mul = -1
age='3m'
add_ptf=True
adj_fut=False
stir_as_swap=True

In [ ]:
options = [
    'RatesTreasuryFuturesOption',
    'RatesShortFuturesOption',
]

future_exclusions = [
    'FxForward',
]

spot_map = {
    'rx': 'GTDEM10Y Govt',
    'sfr': 'SOFRRATE Index',
    'ff': 'FEDL01 Index',
    'ed': 'US0003M Index',
    '0e': 'US0003M Index'   
}

curve_map = {
    'USD CBT 30D': 'USD.OIS',
    'EUR ESTER': 'EUR.ESTER',
    'USD SOFR CME 3M': 'USD.SOFR',
    'USD CME 3M': 'USD.3M'
}

delta_map = {
    'RatesTreasuryFutures': -1,
    'RatesTreasuryFuturesOption': -1,
    'RatesShortFutures': -1, 
    'RatesShortFuturesOption': -1
}

field_map = {
    'rx': 'yld_ytm_mid',
    'tu': 'yld_ytm_mid',
    'fv': 'yld_ytm_mid',
    'ty': 'yld_ytm_mid',
    'us': 'yld_ytm_mid',
    'du': 'yld_ytm_mid',
    'oe': 'yld_ytm_mid',
    'rx': 'yld_ytm_mid',
    'ub': 'yld_ytm_mid',
    'ik': 'yld_ytm_mid',
}

In [ ]:
options_tickers = data.query(f'TradeType in {options}').drop_duplicates('Underlying')[['Ticker', 'Underlying']]
options_underlying_tickers = bq.bdp(options_tickers['Ticker'], {'OPT_UNDL_TICKER': 'uticker'}).rename({'security': 'Ticker'}, axis=1)
options_underlying_tickers['uticker'] = options_underlying_tickers['uticker'] + ' Comdty'
options_tickers = options_tickers.merge(options_underlying_tickers, on='Ticker')

In [ ]:
d = data.merge(options_tickers.drop('Ticker', axis=1), on='Underlying', how='left')
d = d.replace('', np.nan)
d['uticker'] = d['uticker']
d['TradeType'] = d['TradeType']
d.loc[~d['TradeType'].isin(options), 'uticker'] = d.loc[~d['TradeType'].isin(options), 'Ticker']
future_tickers = d.query(f'TradeType not in {future_exclusions}')['uticker'].drop_duplicates().dropna()
gen_fut = get_generic_future(future_tickers, bq=bq);
future_tickers = future_tickers.to_frame().assign(
    gticker=gen_fut,
    cticker=[get_bbg_fut_chain_ticker(fut, days=10, yk=None) for fut in gen_fut] if adj_fut else gen_fut,
)
d = d.merge(future_tickers, on='uticker', how='left')

2023-01-13 03:26:48,656 - Response for 'RX21 Comdty' contains securityError {'securityError': {'source': '19412:rsfrdsvc3', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19412]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-13 03:26:48,657 - Response for 'RX22 Comdty' contains securityError {'securityError': {'source': '19412:rsfrdsvc3', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19412]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-13 03:26:48,658 - Response for 'RX23 Comdty' contains securityError {'securityError': {'source': '19412:rsfrdsvc3', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19412]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-13 03:26:48,659 - Response for 'RX24 Comdty' contains securityError {'securityError': {'source': '19412:rsfrdsvc3', 'code': 43, 'category': 'BAD_SEC', 'message': 'Unknown/Invalid Security  [nid:19412]', 'subcategory': 'INVALID_SECURITY'}}
2023-01-13 0

In [ ]:
def get_future_root(ticker):
    if not pd.isnull(ticker):
        return ticker.split(' ')[0].strip()[:-2].lower()
    return np.nan


def get_aged_futures(uticker, gticker):   
    if not pd.isnull(gticker):
        root = uticker.split(' ')[0].strip()[:-2]
        end = gticker.split(' ', 1)[-1].strip()
        n = int(gticker.split(' ')[0].strip()[len(root):])
        if n > 1:
            aged = f"{root}{n-1} {end}"
        else:
            aged = spot_map[root.lower()]
    else:
        aged = np.nan
    return aged


def get_swap_curve(underlying):
    return curve_map.get(underlying, np.nan)


def get_hpstring(
    instrument,
    curve,
    period, 
    ticker,
    root,
    stir_as_swap=True
):
    if instrument in ['RatesTreasuryFutures', 'RatesTreasuryFuturesOption']:
        field = 'YLD_YTM_MID' # field_map.get(root)
        return f'[BBG {ticker} {field}]'
    elif instrument in ['RatesShortFutures', 'RatesShortFuturesOption']:
        if stir_as_swap:
            return f'[s {curve} {period}]'
        return f'(100 - [BBG {ticker}])' if ticker not in spot_map.values() else f'[BBG {ticker}]'
    elif instrument in ['RatesSwap']:
        return f'[s {curve} {period}]' 

In [ ]:
d['rticker'] = d.apply(lambda row: get_future_root(ticker=row['uticker']), axis=1)
d['cticker_aged'] = d.apply(lambda row: get_aged_futures(uticker=row['uticker'], gticker=row['cticker']), axis=1)
d['hpcurve'] = d.apply(lambda row: get_swap_curve(underlying=row['Underlying']), axis=1)
d['hpperiod'] = d.apply(lambda row: parse_swap_periods(start_date=row['StartDate'], end_date=row['MaturityDate']), axis=1)
d['hpperiod_aged'] = d.apply(lambda row: parse_swap_periods(start_date=row['StartDate'], end_date=row['MaturityDate'], age='3m'), axis=1)
# d['delta_rate'] = d.apply(lambda row: row['Delta'] * delta_map.get(row['TradeType'], 1), axis=1)
d['delta_rate'] = d['Delta']

In [ ]:
d['string'] = d.apply(lambda row: get_hpstring(
    instrument=row['TradeType'],
    curve=row['hpcurve'],
    period=row['hpperiod'],
    ticker=row['cticker'],
    root=row['rticker'],
    stir_as_swap=stir_as_swap
), axis=1)

d['string_aged'] = d.apply(lambda row: get_hpstring(
    instrument=row['TradeType'],
    curve=row['hpcurve'],
    period=row['hpperiod_aged'],
    ticker=row['cticker_aged'],
    root=row['rticker'],
    stir_as_swap=stir_as_swap
), axis=1)

In [ ]:
d_ = d.query('Category == "rates"').groupby(['Strategy', 'string', 'string_aged'])['delta_rate'].sum().reset_index()
d_ = d_.loc[~d_['delta_rate'].between(-delta_tol, delta_tol), :]
d_['Strategy'] =  d_['Strategy'].str.split(':', n=1).str[1:].str[0].str.replace(' ', '_').str.lower()
d_['delta_rate'] = d_['delta_rate'] * delta_mul

In [ ]:
bpv = 1e4

parsed = {}
round_ = 1
for index, group in d_.groupby('Strategy'):
    s_base = (group['delta_rate'] / bpv).apply(lambda x: format(x, f'+.{round_}f')) + '*' 
    s = f"{' '.join(s_base + group['string'])}"
    sa = f"{' '.join(s_base + group['string_aged'])}"
    parsed[index] = s
    parsed[f"{index}A"] = sa

parsed = pd.Series(parsed, dtype=object)

if add_ptf:
    ptf = pd.Series(
        [
            ' + '.join(parsed.index[[*range(0, len(parsed), 2)]]),
            ' + '.join(parsed.index[[*range(1, len(parsed), 2)]])
        ],
        index=['ptf', 'ptfA']
    )
    parsed = pd.concat([parsed, ptf])
parsed.to_clipboard(header=False)

In [ ]:
clarion.metrics()["positions_metrics"][:30]

In [ ]:
clarion.price_metrics().sort_values("metricType")[100:]

In [ ]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
pv = clarion.metric("pv", "PresentValue", "Any")
strike = clarion.metric("strike", "Strike", "Any")
maturity = clarion.metric("maturity_date", "MaturityDate", "Rates", {"Type": "Bumped"})
start = clarion.metric("start_date", "StartDate", "Rates", {"Type": "Bumped"})

fx_price = clarion.metric("Price", "Price", "Fx", {"Unit": "PercentLocal"})
atm_vol = clarion.metric("AtmVol", "Volatility", "Fx", {"Type": "ATM"})
strike_vol = clarion.metric("StrikeVol", "Volatility", "Fx", {"Type": "Strike"})
carry_t1 = clarion.metric(
    "T+1Carry", "Carry", "Fx", {"Period": "1", "StartBumpDays": "1"}
)


fwd_rate = clarion.metric("fwd_rate", "Forward", "Rates")
strike_vol = clarion.metric(
    "strike_vol", "Volatility", "Rates", {"Type": "Strike", "Period": "Daily"}
)

# m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
# m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [pv, fx_price, atm_vol, strike_vol, carry_t1]
rates_metrics = [start, maturity, pv, strike, fwd_rate, strike_vol]
# eq_metrics = [m1,m8,m9]

## Find which products are supported and required parameters

In [ ]:
instrument_types = clarion.price.instrument_types()
instrument_types;

In [ ]:
clarion.price.instrument_type_params("FxVanilla")

### Full Python Code - Rates Swap pricing

In [ ]:
tenors = [
    "o/n",
    "1m",
    "3m",
    "6m",
    "1y",
    "2y",
    "3y",
    "4y",
    "5y",
    "6y",
    "7y",
    "8y",
    "9y",
    "10y",
    "12y",
    "15y",
    "20y",
    "25y",
    "30y",
    "40y",
    "50y",
]

In [ ]:
fwd_starts = [
    None,
    "1m",
    "3m",
    "6m",
    "1y",
    "2y",
    "3y",
    "4y",
    "5y",
    "6y",
    "7y",
    "8y",
    "9y",
    "10y",
]

In [ ]:
offsets = ["1m", "3m", "6m", "1y"]

In [ ]:
swap_floating = "SOFR A"
ccy = "USD"

In [ ]:
trade1 = clarion.instruments.FxVanilla(
    currency_pair="EURUSD", expiry="3M", strike="a", option_type="C"
)

In [ ]:
clarion.price.run(
    fx_metrics,
    trade1.trade,
    start_date="2018-01-01",
    end_date="2022-12-13",
    market_mode="live",
)

In [ ]:
df

In [ ]:
clarion.price.create_trade(
    "RatesSwap",
)

In [ ]:
swap_trades = []
for start in fwd_starts:
    for tenor in tenors:
        trade = clarion.instruments.RatesSwap(
            currency=ccy,
            instrument_name=swap_floating,
            strike="a",
            tenor=tenor,
            start_date=start,
            swap_type="Pay",
            # notional='d20000',
            clearing_house="LCH",
            id=f"{start}x{tenor}" if start else f"spotx{tenor}",
        )
        swap_trades.append(trade.trade)

In [ ]:
end_date = today()
offset = "1y"

In [ ]:
data_list.__len__()

In [ ]:
df = clarion.runner.run(
    rates_metrics,
    swap_trades,
    market_mode="live",
    start_date=end_date - parse_offset(offset),
    end_date=end_date,
    # system_time=system_time,
    # polling_count=10,
    # polling_interval=1
)

In [ ]:
df

In [ ]:
clarion.runner._reqs[0]

In [ ]:
clarion.price.run(

In [ ]:
len(swap_trades)

In [ ]:
res = clarion.price.retrieve(req["KEY"], polling_count=10, polling_interval=1)

In [ ]:
req

In [ ]:
res = clarion.price.retrieve(req["KEY"], polling_count=10)

In [ ]:
res.keys()

In [ ]:
res["TIMEOUT"]

In [ ]:
d = res["DFRAME"]

In [ ]:
res = clarion.price.retrieve(req["KEY"])

In [ ]:
res["DFRAME"]

In [ ]:
def run_multiple();

In [ ]:
swap_curves = res["DFRAME"].sort_values(["period", "start_date", "maturity_date"])
periods = swap_curves.id.str.split("x")
swap_curves = swap_curves.assign(
    start=periods.str[0],
    tenor=periods.str[1],
    date=pd.to_datetime(swap_curves["period"]),
)

In [ ]:
date = today()

In [ ]:
swap_curves.query(f"date=='{date:%Y-%m-%d}'").plot(
    x="tenor",
    y="strike",
    color="start",
    title=swap_floating,
    height=500,
    template="plotly_white",
)

In [ ]:
fra_floating = "LIBOR 3M"
ccy = "USD"

In [ ]:
fra_trades = []
for start in fwd_starts:
    trade = clarion.instruments.RatesFra(
        currency=ccy,
        instrument_name=fra_floating,
        strike="a",
        start_date=start,
        fra_type="Pay",
        # notional='d20000',
        clearing_house="LCH",
        id=f"{start}x3m" if start else f"spotx3m",
    )
    fra_trades.append(trade.trade)

In [ ]:
offsets = ["1m", "3m", "6m", "1y"]

In [ ]:
end_date = today()
offset = "1y"

In [ ]:
response = clarion.price.run(
    rates_metrics,
    fra_trades,
    market_mode="live",
    start_date=f"{end_date - parse_offset(offset):%Y-%m-%d}",
    end_date=f"{end_date:%Y-%m-%d}",
)

In [ ]:
fra_curves = response["DFRAME"].sort_values(["period", "start_date", "maturity_date"])
periods = fra_curves.id.str.split("x")
fra_curves = fra_curves.assign(
    start=periods.str[0], tenor=periods.str[1], date=pd.to_datetime(curves["period"])
)

In [ ]:
fra_curves

In [ ]:
" " + fra_curves.query(f"date=='{date:%Y-%m-%d}'")["start"]

In [ ]:
fra_curves.query(f"date=='{date:%Y-%m-%d}'").plot(
    x="start", y="strike", title=floating, height=500, template="plotly_white"
)

### Full Python Code - Rates swaption pricing

In [ ]:
rates_swaption = clarion.create_trade(
    "swaption_1",
    "ratesSwaption",
    {
        "InstrumentName": "BBSW Q 3M",
        "Currency": "AUD",
        "Expiry": "10-Mar-23",
        "Notional": 100000000,
        "Tenor": "6M",
        "Strike": "0.2",
        "SwaptionType": "Pay",
        "CollateralCurrency": "USD",
        "SettleType": "Physical Cleared LCH",
        "PaySettle": "Spot",
    },
)

response = clarion.price.run(
    rates_metrics,
    rates_swaption,
    start_date="2022-03-01",
    end_date="2022-03-16",
    market_mode="reval",
)
if response["ERROR"]:
    print(response["ERROR"])
    print(response["IS_ERROR"])
    print(response["HAS_ERROR"])
else:
    display(response["DFRAME"])

### Full Python Code - Equity future option pricing

In [ ]:
equity_futOpt = clarion.create_trade(
    "id2",
    "equityFutureOption",
    {
        "InstrumentName": "AS51 SFE",
        "UnderlyingExpiry": "Sep23",
        "ContractType": "OTC",
        "Expiry": "2023-09-21",
        "Strike": "5500",
        "OptionType": "C",
        "ExerciseType": "A",
        "Quantity": 100,
    },
)
response = clarion.price.run(
    eq_metrics,
    equity_futOpt,
    start_date="2022-06-01",
    end_date="2022-09-01",
    market_mode="reval",
)
if response["ERROR"]:
    print(response["ERROR"])
    print(response["IS_ERROR"])
    print(response["HAS_ERROR"])
else:
    display(response["DFRAME"])

## Retrieve Exchange close pricing
#### Exchange close pricing is available back up to **4w history** from todays date

In [ ]:
rates_STIRFut = clarion.create_trade(
    "stir1",
    "ratesShortTermFuture",
    {"InstrumentName": "EURIBOR 3M ICE", "Currency": "EUR", "Tenor": "H23"},
)

In [ ]:
m1 = clarion.metric("FwdRate", "Forward", "Rates")
response = clarion.price.run(
    [m1],
    rates_STIRFut,
    start_date="2022-09-01",
    end_date="2022-09-05",
    market_mode="reval",
    settings={"market_name": "exchange"},
)
response["DFRAME"]